# 8. Runnable

Let's step back a little... A [fundamental concept](https://python.langchain.com/v0.2/docs/concepts/) of LangChain is the Runnable interface. This the "chain" in Lang...Chain

The [Runnable](https://python.langchain.com/docs/concepts/runnables/) interface in LangChain provides a consistent and versatile way to interact with components like language models, retrievers, and tools, allowing them to be invoked, batched, streamed, and inspected. 

It enables efficient execution of tasks, supports asynchronous and parallel processing, and facilitates the composition of complex workflows through chaining.

Runnables can also be customized with runtime configurations like concurrency limits, callbacks, and metadata, making them adaptable for diverse applications in AI and machine learning workflows.

![alt text](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*UyVmJ3RFRjXS-czkbjx1MA.png)

In [1]:
# From now I'll use a module to import our llm from tools.py
from tools import llm # containing my ChatOllama
from langchain.chains import TransformChain

In [ ]:
llm.invoke("hello")

In [3]:
# Step 1: Preprocessing (add context)
def add_context(inputs):
    user_input = inputs["input"]
    return {"input": f"Context: You are a helpful assistant. {user_input}"}

# Step 2: Language model invocation (trivia generator)
def generate_response(inputs):
    response = llm.invoke([{"role": "user", "content": inputs["input"]}])
    return {"output": response.content.replace(' ',' ~~ ')}

# Step 3: Post-processing (make result uppercase)
def clean_output(inputs):
    output = inputs["output"]
    return {"final_output": output.upper()}

In [4]:
# We define Runnable Chains that can be executed by the LangChain engine
# Each chain is a sequence of transformations that are applied to the input data
preprocess_chain = TransformChain(
    input_variables=["input"],
    output_variables=["input"],
    transform=add_context,
)
llm_chain = TransformChain(
    input_variables=["input"],
    output_variables=["output"],
    transform=generate_response,
)
postprocess_chain = TransformChain(
    input_variables=["output"],
    output_variables=["final_output"],
    transform=clean_output,
)

In [ ]:
# This is a pipeline that combines "Runnable" elements
pipeline = preprocess_chain | llm_chain | postprocess_chain

# Test the pipeline
user_input = "What is the capital of France?"
result = pipeline.invoke({"input": user_input})
print("Final Output:", result["final_output"])

In LangChain, components process specific input types and produce corresponding outputs. 

Prompts create structured requests, ChatModels and LLMs handle text or messages, and OutputParsers refine results. 

Retrievers fetch documents based on a query, while Tools adapt their inputs and outputs based on their purpose, ensuring smooth and consistent workflows.

| Runnable Component     | Input Type                                  | Output Type               |
|---------------|---------------------------------------------|---------------------------|
| Prompt        | dictionary                                  | PromptValue               |
| ChatModel     | a string, list of chat messages, or a PromptValue | ChatMessage              |
| LLM           | a string, list of chat messages, or a PromptValue | String                   |
| OutputParser  | the output of an LLM or ChatModel           | Depends on the parser     |
| Retriever     | a string                                    | List of Documents         |
| Tool          | a string or dictionary, depending on the tool | Depends on the tool       |